In [1]:
import ee
import geetools
import urllib.request
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VPXxc3EYDiKa2ZrjeNn3dwAyI-Rp34gz3yrBV-rteeg_XqpPBNh9gQ

Successfully saved authorization token.


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions:

https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=X3HdwwlPOnxll13GWExVRdcvW1TK3qQsAuAeNjCqpqU&tc=pxt-Fa2tUQTGmupZziNzUSiAZfEbRhVojCjyKXr_4rY&cc=0nAGl69OyneFzAh-tdyudQyzQ9-agywwqXGUSnH9U_0

The authorization workflow will generate a code, which you should paste in the box below.

In [3]:
ee.Initialize()

## Export Copernicus

In [4]:
lat1 = 37.
lat2 = 41.
lon1 = -102.
lon2 = -109.
longitudes = []
latitudes = []
for i in np.arange(lon2,lon1,0.036):
    for j in np.arange(lat1,lat2,0.036):
        longitudes.append(i)
        latitudes.append(j)
locations = pd.DataFrame({
    'longitude': longitudes,
    'latitude': latitudes
})
locations

,longitude,latitude
0,-109.000,37.000
1,-109.000,37.036
2,-109.000,37.072
3,-109.000,37.108
4,-109.000,37.144
...,...,...
21835,-102.016,40.852
21836,-102.016,40.888
21837,-102.016,40.924
21838,-102.016,40.960


In [ ]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
start_location = 12000 # where you will start your loop based on the dataframe
end_location = 13000 # where you will end your loop based on the dataframe

for index,row in locations.iloc[start_location:end_location].iterrows():
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']
    center = ee.Geometry.Point(longitude,latitude)
    
    # Import Sentinel dataset
    s2 = (ee.ImageCollection("COPERNICUS/S2_SR")   # LANDSAT/LC08/C01/T1_SR or COPERNICUS/S2_SR
      .filterBounds(center)
      .sort('CLOUDY_PIXEL_PERCENTAGE',True)
      .filterDate('2020-05-01', '2020-06-30')
      .first()
    )
    
    # Create coordinates
    try:
        global sentinel_footprint
        sentinel_footprint = (ee.Geometry.Polygon(coords=[(longitude - 0.018,latitude - 0.018),
                                                      (longitude - 0.018,latitude + 0.018),
                                                     (longitude + 0.018,latitude - 0.018),
                                                     (longitude + 0.018,latitude + 0.018),]))
    except:
        print("No Footprint found")
        continue
        
    # Create a rectangular export area
    exportArea = (sentinel_footprint.centroid().buffer(2000).bounds())
    
    # Export Parameters
    s2Vis = {
        'region': exportArea,
        'crs': (s2.select('B4').projection()),
        'dimensions': imageDimensions,
        'format': 'png',
        'min': vis_min,
        'max': vis_max,
        'bands': vis_bands,
        'gamma':1.5
    }
    
    # Export
    s2_url = (s2.getThumbURL(s2Vis))
    urllib.request.urlretrieve(s2_url,f"..\\image{id_}_{round(longitude,2)}_{round(latitude,2)}.png")
    print(f'finished number {id_} at {round(longitude,2)} : {round(latitude,2)}')
    print(s2_url)

finished number 12000 at -105.15 : 37.58
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c7542e8e08846785eee931f045a64f95-185d80aad02d072f923c1c4d89cebfb5:getPixels
finished number 12001 at -105.15 : 37.61
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9b991312952b209327e29c0e7d8ad598-15b8368a21707a515f926b0f4e8634f1:getPixels
finished number 12002 at -105.15 : 37.65
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8432afe27c571b19bd45e311e671e89a-6c8c8c8fbf0d616f438affc506e7438e:getPixels
finished number 12003 at -105.15 : 37.68
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6b250ba457dc3e76acaaac62812c01ed-ad125a5b52e9097add6036a9c0bc8cd2:getPixels
finished number 12004 at -105.15 : 37.72
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/41e02680ddb2128a3a61a2380ce335dd-d294dc0120c4b2380c95ec5aa09add89:getPixels
finis

finished number 12042 at -105.15 : 39.09
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6d8ab3658757f894506e6af0059da636-c4433e09156b81c3a478897f936416a8:getPixels
finished number 12043 at -105.15 : 39.12
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/55aa00b90e15e27698d2eafe4dd3dd39-97d35c9abac38974024ea00516f733ad:getPixels
finished number 12044 at -105.15 : 39.16
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b2e3a511ea98e39ef2f59c61d4c5daac-ec674449d39787240753bf299bff0b69:getPixels
finished number 12045 at -105.15 : 39.2
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/00ea132e17fdaf357c32be81a58e9969-75bb929725781e5500940c1dccb48a43:getPixels
finished number 12046 at -105.15 : 39.23
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1eb9d09f34ca4dcc19aa550ea21feccb-b5ab961e74b53be56307d8ee41849db3:getPixels
finish

finished number 12084 at -105.15 : 40.6
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/274cdcfc835c3c218c217e0f1ae8134f-badb0e505a4c7bf6cbd2515cec396938:getPixels
finished number 12085 at -105.15 : 40.64
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/808af15295dfcdd3d54c43b446c8b90f-98458371321040e65a9b8b98bca0fd8a:getPixels
finished number 12086 at -105.15 : 40.67
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/61e76ee8c77adeb8c4c8833f1e2f8bf9-734732dde12773f9ec973a5fa3f13d51:getPixels
finished number 12087 at -105.15 : 40.71
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1b8b8cf92ef4e830abbe6c036563ea39-af21495a25774efa76d86a73316c7d42:getPixels
finished number 12088 at -105.15 : 40.74
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4439a71673db5da35db7b7f0303d1e95-01ba6a1a8673d45280ce0f10808627c8:getPixels
finish

finished number 12126 at -105.11 : 38.08
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/cd768683553483837451dd1db628f5df-bd1d77b9bfaf92320d7f9a3466d78a23:getPixels
finished number 12127 at -105.11 : 38.12
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0d19dd5034578de67d40edb34635fb7d-571463bb6992b3f418ed3ab2ce02cd60:getPixels
finished number 12128 at -105.11 : 38.15
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ed2557fb5061ed6b4db81b21b3cef0db-8aabdc0962c24ecb2a73bdec401d0346:getPixels
finished number 12129 at -105.11 : 38.19
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/806770601d33407b206ee7eb84ea8cc9-0392dc28121611ba8871556219decc74:getPixels
finished number 12130 at -105.11 : 38.22
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/11154bfa60f57112a0eb0992a07a6baf-cece6b026dcf169d2f8dbc20c5a4377c:getPixels
finis

finished number 12168 at -105.11 : 39.59
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e1eb7fa12e3b036c7dfb8feed00a7e71-dff0930676db7f097047efca3502cbf3:getPixels
finished number 12169 at -105.11 : 39.63
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7c43cf48aa2d7203e749fee9252c3d4f-0596c875d9d9112cd8d9325dfe4119ca:getPixels
finished number 12170 at -105.11 : 39.66
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/888cacad5cc9689e3f4d19364aabf921-73818aaef63a97db57d394a9c1aee0a5:getPixels
finished number 12171 at -105.11 : 39.7
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7969abaec2f845de996a7b2682faa22a-9ac4aeb51aec0faa7c0ab7fab7a1b28a:getPixels
finished number 12172 at -105.11 : 39.74
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f8be0313baa6a1d2c7a5e2c4f1532e16-c52cd2e8fda1d9b1acdf8217b4b8189c:getPixels
finish

...

finished number 12378 at -105.04 : 39.09
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/3f4c6742acafeb8836e3273e4980cca9-652381588929ebc1f7e68dfd7a592730:getPixels
finished number 12379 at -105.04 : 39.12
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/75879b94384629c00317ff3ce6417521-9fb00c98e4b97b458da4998759961987:getPixels
finished number 12380 at -105.04 : 39.16
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e97fe9919ffe2566f837e8506dfe271c-b83d76112fb3ccda3ea47b11e3142d1e:getPixels
finished number 12381 at -105.04 : 39.2
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0f612869288af99ae283d9f219521818-9ba22c6e6733847ae0a9c300e4267304:getPixels
finished number 12382 at -105.04 : 39.23
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6dcfa5f7de4fe497155a7e88e861798b-970c0ad0e05387957d11dbd9ecc5ccd4:getPixels
finish

finished number 12420 at -105.04 : 40.6
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/aeefc71d4eb260a908bd0a40e8f10b94-5dac0f9bc3abd2afddaf69e55a4019e8:getPixels
finished number 12421 at -105.04 : 40.64
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0b1c29c15b273ae93c193fac96602adf-2667afc0aecdda9a2a450e1067106d98:getPixels
finished number 12422 at -105.04 : 40.67
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/544c84aff1553e090532540d15fe45db-bcf55bb350071517d5ab4428b2f61558:getPixels
finished number 12423 at -105.04 : 40.71
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/dc467f0a7d51fbf2d3eee0ff63e1e6f7-fc7f61db549ca03074f2dda3a7077572:getPixels
finished number 12424 at -105.04 : 40.74
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8eb91f41f1ce96fe919f1a46153bfe55-e28bb3bb093f1a0aa38d51a3c0303396:getPixels
finish

finished number 12462 at -105.0 : 38.08
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0e8501961474e8fdef064359f33bd304-d4784f078cc0e88243a4b5eb5986d66c:getPixels
finished number 12463 at -105.0 : 38.12
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/521328474d71f48526c387e288ea3e15-4a7eaeab3380e6a287c5b78ff8083905:getPixels
finished number 12464 at -105.0 : 38.15
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/04fa710e56255481bf0c350b498d3ab3-c5ea990226c0b6ad8ec1b7a4defe2a2e:getPixels
finished number 12465 at -105.0 : 38.19
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/17db59e45a7d84f9f080cc8184e959ae-71048eaaa67acf9620100e4253853c22:getPixels
finished number 12466 at -105.0 : 38.22
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/73a5cbf794af4ae5415ed0ea4af93a69-e715b5595ba06c8918837a399101af57:getPixels
finished n

finished number 12504 at -105.0 : 39.59
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/230a5d6e13a025f7641115f0deedbcba-c82c960d28038079d20cd1425d239390:getPixels
finished number 12505 at -105.0 : 39.63
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d5777fa80783c248ef9d8d758fa37359-51c574ebdcc4c02df9b87ad29a117fed:getPixels
finished number 12506 at -105.0 : 39.66
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fcc8648c7a52a7de89530ca4c272a07b-fa777637922aea7bfc656b6f27ed6e10:getPixels
finished number 12507 at -105.0 : 39.7
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0c15bd3b946e75946a6fa396a1e1bfbc-76f6098257e9ab362ff84ec0102ee5b3:getPixels
finished number 12508 at -105.0 : 39.74
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/1c4744b2232d8b21b29093acab187dd5-ed89c1c32dd6839f5207afe2b68dee9d:getPixels
finished nu

finished number 12546 at -104.97 : 37.07
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/02c149bcf027233fbe4cdbb8728d8d85-f695fc90368a7d3ce57cf9f99a4da478:getPixels
finished number 12547 at -104.97 : 37.11
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b95d6ffa7a1ef08ac551c5be3ba18ef2-3f151d6a60a3591cbce13baea574f3eb:getPixels
finished number 12548 at -104.97 : 37.14
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/ca39c8d23638f490191cc603a85ab8e3-1a7e1b9ae939673642914d9c26d3c653:getPixels
finished number 12549 at -104.97 : 37.18
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/7eeb815c6f7c1f0c1775b0666199f804-cb1da5c5f3b943ec29be2de5eb1e121b:getPixels
finished number 12550 at -104.97 : 37.22
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/69724f6b7dd3967a6a6f1aa6db3e8d34-9d59bf97737a275d1191355d54a71923:getPixels
finis

## Export WorldCover (not functional yet)

In [ ]:
lat1 = 37.
lat2 = 41.
lon1 = -102.
lon2 = -109.
longitudes = []
latitudes = []
for i in np.arange(lon2,lon1,0.036):
    for j in np.arange(lat1,lat2,0.036):
        longitudes.append(i)
        latitudes.append(j)
locations = pd.DataFrame({
    'longitude': longitudes,
    'latitude': latitudes
})
locations

In [ ]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['Map'] #Includes the bands for RGB
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
start_location = 200 # where you will start your loop based on the dataframe
end_location = 1000 # where you will end your loop based on the dataframe

for index,row in locations.iloc[start_location:end_location].iterrows():
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']
    center = ee.Geometry.Point(longitude,latitude)
    
    # Import Sentinel dataset
    wc = (ee.ImageCollection("ESA/WorldCover/v200")   # LANDSAT/LC08/C01/T1_SR or COPERNICUS/S2_SR
      .filterBounds(center)
      .filterDate('2020-05-01', '2020-06-30')
      .first()
    )
    
    # Create coordinates
    try:
        global sentinel_footprint
        sentinel_footprint = (ee.Geometry.Polygon(coords=[(longitude - 0.018,latitude - 0.018),
                                                      (longitude - 0.018,latitude + 0.018),
                                                     (longitude + 0.018,latitude - 0.018),
                                                     (longitude + 0.018,latitude + 0.018),]))
    except:
        print("No Footprint found")
        continue
        
    # Create a rectangular export area
    exportArea = (sentinel_footprint.centroid().buffer(2000).bounds())
    
    # Palette
    
    palette = [
        '006400', #Tree cover
        'ffbb22', #Shrubland
        'ffff4c', #Grassland
        'f096ff', #Cropland
        'fa0000', #Built-up
        'b4b4b4', #Bare / sparse vegetation
        'f0f0f0', #Snow and ice
        '0064c8', #Permanent water bodies
        '0096a0', #Herbaceous wetland
        '00cf75', #Mangroves
        'fae6a0', #Moss and lichen
    ]
    
    joined_palette = ','.join(palette)
    print(joined_palette)
    
    # Export Parameters
    wcVis = {
        'region': exportArea,
        'crs': (wc.select('Map').projection()),
        'dimensions': imageDimensions,
        'format': 'png',
        'min': vis_min,
        'max': vis_max,
        'bands': vis_bands,
        'gamma':1.5,
        'palette': joined_palette
    }
    
    # Export
    wc_url = (wc.getThumbURL(wcVis))
    urllib.request.urlretrieve(wc_url, fr"..\raw_data\image{id_}_{round(longitude,2)}_{round(latitude,2)}.png")
    print(f'finished number {id_} at {round(longitude,2)} : {round(latitude,2)}')

In [ ]:
'10', '20', '30', '40', '50', '60', '70', '80', '90', '95', '100'